# Consistent Bayes: Exponential Decay Example (Parallelized)
---

Copyright 2018 Michael Pilosov


### Import Libraries


In [1]:
from matplotlib import pyplot as plt
import numpy as np

In [3]:
import ipyparallel as ipp # import the client
rc = ipp.Client()
dv = rc[:]
rc.ids # list the ids of the engine the client can communicate with

/home/mpilosov/anaconda3/envs/test_py_env/lib/python3.6/site-packages/ipyparallel/client/client.py:458: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@math-ws-204')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


[0, 1, 2, 3, 4, 5, 6, 7]

In [4]:
%%px
import numpy as np
from cbayes import sample, solve, distributions
lambda_ref = np.array([[0.5, 0.5]])

def MSE_generator(model, obs_data, sigma=1):   # this generates a sum of squared residuals.
    def QoI_fun(inputs): # that conforms to our desired model input
        M = len(obs_data)
        predictions = model(inputs)
        assert predictions.shape[1] == M
        residuals = predictions - obs_data
        QoI = (1./M)*np.sum( (residuals/sigma)**2, axis=1 )
        return QoI
    return QoI_fun

# The model of the harmonic oscillator 
def makemodel(t):
    def model(lam = lambda_ref ):
#         return np.cos(lam[:,1]*t + np.arccos(lam[:,0]) )
        QoI = lam[:,0].reshape(-1,1)*np.exp(np.outer(lam[:,1], -t))
        if QoI.shape[0] == 1:
            return QoI.ravel() # this allows support for simpler 1D plotting.
        else:
            return QoI
    return model


In [7]:
num_observations = np.array([1])# 2, 5, 10, 25, 50])
true_observational_variance = np.array([0.0125, 0.025, 0.05]) 
start_time = np.array([0.25, 0.5, 0.75, 1])
obs_len = np.array([1, 2 , 3, 4])

In [8]:
S = np.array([ num_observations, true_observational_variance, start_time, obs_len])
num_iterations = np.prod([ S[i].shape[0] for i in range(S.shape[0]) ])
SS = np.array( np.meshgrid(*S) )
trial_vector = SS.ravel().reshape(S.shape[0],num_iterations).T
# print(trial_vector[0:5,:])
print("Number of total trials:", num_iterations) 

Number of total trials: 48


# Example Summary

---

The second example is defined by the model for a harmonic oscillator given by the initial value problem  

\begin{equation}
	\begin{cases}
		\frac{d^2u}{dt^2} &= -\lambda_2^2 u, \ t>0, \\
		u(0) &= \lambda_1. 
	\end{cases}
\end{equation}

Choosing a particular parameter $\lambda=(\lambda_1,\lambda_2)\in\Lambda\subset[-1,1]\times\mathbb{R}^+$ corresponds to fixing an initial condition, $\lambda_1\in[-1,1]$, and frequency of oscillation, $\lambda_2$, where we again assume that $\lambda_2>0$.

The solution to the above is
\begin{equation}
	u(t) = \cos(\lambda_2 t + \arccos(\lambda_1)).
\end{equation}

---
## Define Functions

In [11]:
%%px
mins = np.array([0, 0]) # original domain
maxs = np.array([1, 1])
nbins = 250
def grid(nbins=5, mins=np.zeros(1), maxs=np.ones(1) ):
    dim = len(mins)
    S = np.array( np.meshgrid(*[np.linspace(i,j,nbins) for i,j in zip(mins+1./(2*nbins), maxs-1./(2*nbins))]) )
    SS = S.ravel().reshape(dim,nbins**dim).T
    return SS

X = grid(nbins, mins, maxs)

def run_trial(trial):
    num_samples = int(1E4)
    num_observations, sd_true, start_time, obs_length = trial
    end_time = start_time + obs_length
    sd_guess = sd_true
    num_observations = int(num_observations)
    num_samples = int(num_samples)
    
    t = np.linspace(start_time, end_time, num_observations)
    model = makemodel(t)
    u_true = model()
    obs_data = u_true + np.random.randn(int(num_observations))*sd_true
    PtO_fun = MSE_generator(model, obs_data, sd_guess)
    
    s_set = sample.sample_set(size=(num_samples, 2))
    s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': 0, 'scale': 1}, dim=0) # initial condition
    s_set.set_dist('beta', kwds={'a': 1, 'b': 1, 'loc': 0, 'scale': 1}, dim=1) # frequency
    s_set.generate_samples()
    p_set = sample.map_samples_and_create_problem(s_set, PtO_fun)
    p_set.compute_pushforward_dist()
    p_set.set_observed_dist('gamma', {'a':num_observations/2, 'scale':2/num_observations}, dim=0)
    p_set.set_ratio()
    #     solve.problem(p_set) # if you care to perform accept/reject on the prior, uncomment this.
#     accept_inds = p_set.accept_inds
#     lam_accept = p_set.input.samples[accept_inds,:]
#     num_accept = len(accept_inds)
    print('trial =', trial)
#     print('Number accepted: %d = %2.2f%%'%(num_accept, 100*np.float(num_accept)/num_samples))
    
    # Perform accept/reject on that grid you see plotted above.
    Y = PtO_fun(X)
    inds = solve.perform_accept_reject(X, p_set.compute_ratio(Y))
    abs_error = [*np.abs( lambda_ref[0,:] - np.mean(X[inds,:],axis=0) )]
    rel_error = [*np.abs( p_set.observed_dist.mean()-np.mean(Y[inds],axis=0))/p_set.observed_dist.mean() , *np.abs( p_set.observed_dist.std()-np.std(Y[inds],axis=0))/p_set.observed_dist.std()]
#     print('Abs Error in Parameter and MAP:', abs_error )
#     print('Param. Test Stats for Gamma Dist:', rel_error)
#     print(' ')
    
    post_eval = p_set.evaluate_posterior(X)
    return {'p': post_eval, 'a': abs_error, 'r': rel_error, 'n': len(inds), 't': trial}


In [12]:
dv.scatter('trial_chunk', trial_vector)

<AsyncResult: scatter>

In [13]:
%%capture output
%%px
D = list(map(run_trial, trial_chunk))

In [14]:
output.show()

[stdout:0] 
trial = [1.     0.0125 0.25   1.    ]
trial = [1.     0.0125 0.25   2.    ]
trial = [1.     0.0125 0.25   3.    ]
trial = [1.     0.0125 0.25   4.    ]
trial = [1.     0.0125 0.5    1.    ]
trial = [1.     0.0125 0.5    2.    ]
[stdout:1] 
trial = [1.     0.0125 0.5    3.    ]
trial = [1.     0.0125 0.5    4.    ]
trial = [1.     0.0125 0.75   1.    ]
trial = [1.     0.0125 0.75   2.    ]
trial = [1.     0.0125 0.75   3.    ]
trial = [1.     0.0125 0.75   4.    ]
[stdout:2] 
trial = [1.     0.0125 1.     1.    ]
trial = [1.     0.0125 1.     2.    ]
trial = [1.     0.0125 1.     3.    ]
trial = [1.     0.0125 1.     4.    ]
trial = [1.    0.025 0.25  1.   ]
trial = [1.    0.025 0.25  2.   ]
[stdout:3] 
trial = [1.    0.025 0.25  3.   ]
trial = [1.    0.025 0.25  4.   ]
trial = [1.    0.025 0.5   1.   ]
trial = [1.    0.025 0.5   2.   ]
trial = [1.    0.025 0.5   3.   ]
trial = [1.    0.025 0.5   4.   ]
[stdout:4] 
trial = [1.    0.025 0.75  1.   ]
trial = [1.    0.025 0.75 

In [ ]:
Dg = np.array(list(dv.gather('D').get() ))
# np.save("ED_savefile_gridsearch_allruns_fine.npy", Dg)
# np.save("savefile_gridsearch_var_window.npy", Dg)